import

In [28]:
import os

from langchain.chains import ConversationChain, LLMChain, LLMRouterChain
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate

xionic-ko-llama

In [27]:
from openai import OpenAI

client = OpenAI(
    base_url = "http://sionic.chat:8001/v1",
    api_key = "934c4bbc-c384-4bea-af82-1450d7f8128d"
)

In [7]:
response = client.chat.completions.create(
    model="xionic-ko-llama-3-70b",
    messages=[
        {"role": "system", "content": "You are an AI assistant. You will be given a task. You must generate a detailed and long answer in korean."},
        {"role": "user", "content": "아이브 리더가 활동했던 이전 걸그룹의 명칭은?"}
    ]
)

print (response)

ChatCompletion(id='cmpl-f0f3875f4a9043dba8d47eb2aedb0079', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='안녕하세요. 아이브(IVE)의 리더, 안유진은 이전에 걸그룹 Iz*One의 멤버로 활동했습니다. Iz*One은 2018년 Mnet의 프로듀스 48을 통해 결성된 프로젝트 걸그룹으로, 총 12명의 멤버로 구성되어 있었습니다. 이 그룹은 2019년 2월 6일에 데뷔 싱글 "라비앙로즈"를 발매하면서 활동을 시작했으며, 2021년 4월 29일에 공식적으로 해체되었습니다. 안유진은 Iz*One에서 리더를 맡아 활동했으며, 그룹의 해체 후 현재는 아이브의 리더로 활동하고 있습니다.', role='assistant', function_call=None, tool_calls=None), stop_reason=None)], created=1721196750, model='xionic-ko-llama-3-70b', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=162, prompt_tokens=56, total_tokens=218))


문서 가져오기

In [36]:
PATH = "C:/Users/SUNJIN/Documents/인턴/03_ideachatbot/data"
Product_data = os.path.join(
    PATH, "년도별 신제품 리스트_냉장-240425.xlsx"
)

In [34]:
def read_prompt_template(file_path: str) -> str:
    with open(file_path, "rb") as f:
        prompt_template = f.read()

    return prompt_template

In [41]:
def create_chain(template_path):
    prompt_template = read_prompt_template(template_path)
    
    chain = LLMChain(
        llm="xionic-ko-llama-3-70b",
        prompt=ChatPromptTemplate.from_template(template=prompt_template),
        max_tokens=150,
        temperature=0.7,
        stop=None
    )
    
    return chain

In [42]:
product_data = create_chain(
    template_path=Product_data
)

# 엑셀을 읽을 수 있는 형식으로

TypeError: expected str, got bytes